In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dirpath = 'drive/MyDrive/vision project/xmls'  # The directory where the xml file was originally stored
frame_dirpath = 'drive/MyDrive/vision project/frames/'

Mounted at /content/drive


# `load database`

In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import cv2
import numpy as np
dirpath = 'drive/MyDrive/vision project/xmls'  # The directory where the xml file was originally stored
# newdir = 'E:/univarsity/machine vision/final_project/frames/New Folder'  # edit the txt directory formed after the label
player_label=np.array([])
player_image=np.array([])
a_3d_array = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
j=0
for fp in os.listdir(dirpath):
    root = ET.parse(os.path.join(dirpath, fp)).getroot()

    xmin, ymin, xmax, ymax = 0, 0, 0, 0
    sz = root.find('size')
    width = float(sz[0].text)
    height = float(sz[1].text)
    filename = root.find('filename').text
    im = Image.open(frame_dirpath + filename)
    img = cv2.imread(frame_dirpath + filename)
    i=0
    for child in root.findall('object'):  # Found all the boxes in the image
        #make image array
        sub = child.find('bndbox')  # Found the label value of the box and read it
        xmin = float(sub[0].text)
        ymin = float(sub[1].text)
        xmax = float(sub[2].text)
        ymax = float(sub[3].text)
        x_center = (xmin + xmax) / (2 * width)
        y_center = (ymin + ymax) / (2 * height)

        im1 = im.crop((xmin, ymin, xmax, ymax))
        cropped_image = img[int(ymin):int(ymax), int(xmin):int(xmax)]
        dim=(100,100)
        resized = cv2.resize(cropped_image, dim, interpolation=cv2.INTER_AREA)

        if j>1:
            player_image = np.concatenate((player_image,  [resized]), axis=0)
        if j == 1:
            player_image = np.stack((player_image, resized))
            j = j + 1
        if j == 0:
            player_image = resized
            j = j + 1

        name = root.find('name')
        if child[0].text=='red':
            player_label=np.append(player_label,0)
        if child[0].text=='blue':
            player_label=np.append(player_label,1)
        if child[0].text=='yellow':
            player_label=np.append(player_label,2)
    player_image=np.array(player_image)
print(player_image.shape)
print(player_label.shape)
np.save("player_image.npy", player_image)
np.save("player_label.npy", player_label)

# load libraries and seprate test and train data


In [ ]:
# %tensorflow_version 1.x
import cv2
import numpy as np
import tensorflow as tf

from keras.datasets import mnist
from keras.layers import Input ,Dense,Activation, Conv2D,AveragePooling2D,Flatten
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

In [ ]:
print("player_image shape is {}".format( player_image.shape ))
print("player_label shape is {}".format( player_label.shape ))

x_train, x_test, y_train, y_test  = train_test_split(player_image, player_label , test_size=0.1, shuffle=True, random_state=5)
print("train_x shape is {}".format( x_train.shape ))
print("train_y shape is {}".format( y_train.shape ))
print("test_x shape is {}".format( x_test.shape ))
print("test_y shape is {}".format( y_test.shape ))

player_image shape is (1073, 100, 100, 3)
player_label shape is (1073,)
train_x shape is (965, 100, 100, 3)
train_y shape is (965,)
test_x shape is (108, 100, 100, 3)
test_y shape is (108,)


# normalize input image of x_train and x_test

In [ ]:
image_size = x_train.shape[0]
# print(x_train)
x_train = x_train/255
x_test = x_test/255
print(f"max value of x is {x_train.max()} and min value is {x_train.min()} ")
print(f"unique value of y_train {np.unique(y_train)}")


max value of x is 1.0 and min value is 0.0 
unique value of y_train [0. 1. 2.]


# categorical class label
we use label to create matrix for them 
it mean get value integer that is label of class and make it vector that spcial component of this vector is 1 for spcial class number 

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print("y_train shape after categorical it")

y_train shape after categorical it


# make datagenerator 
datagenerator make diffrent type of data from our data by change angle of image by rotation or shifting image to right or left or zomming or fliping or ...

In [ ]:
datagen = ImageDataGenerator(
          rotation_range=30,
          width_shift_range=0.2,
          height_shift_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')

# build and compile cnn model 

In [ ]:
def build_model(input_shape):
  
  x_input = Input(shape =input_shape,name = 'input')

  x = Conv2D(filters = 16,kernel_size = (2,2), strides = 1, padding = 'valid',name = 'conv2')(x_input)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size =2,strides = 2,name = 'pad2')(x)

  x = Conv2D(filters = 8,kernel_size = (2,2), strides = 1, padding = 'valid',name = 'conv2a')(x_input)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size =2,strides = 2,name = 'pad2')(x)

  x = Flatten()(x)

  x = Dense(units = 120, name = 'fc_1')(x)

  x = Activation('relu', name = 'relu_1')(x)
  # x = Dropout(rate = 0.5)

  x = Dense(units = 84, name = 'fc_2')(x)
  x = Activation('relu', name = 'relu_2')(x)
  # x = Dropout(rate = 0.5)


  outputs = Dense(units = 3,name = 'softmax', activation='softmax')(x)
  
  model = Model(inputs = x_input, outputs = outputs)
  model.summary()

  return model

In [ ]:
model = build_model(input_shape=(100,100,3))
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy' ,metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 100, 100, 3)]     0         
                                                                 
 conv2a (Conv2D)             (None, 99, 99, 8)         104       
                                                                 
 activation_1 (Activation)   (None, 99, 99, 8)         0         
                                                                 
 pad2 (AveragePooling2D)     (None, 49, 49, 8)         0         
                                                                 
 flatten (Flatten)           (None, 19208)             0         
                                                                 
 fc_1 (Dense)                (None, 120)               2305080   
                                                                 
 relu_1 (Activation)         (None, 120)               0     

# fit and train our model 
now by using our train dataset fit our model to train dataset and train on that dataset

In [ ]:
# model.fit(x = x_train, y= y_train , batch_size=5, epochs = 4)
batch_size = 64
H = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
  validation_data=(x_test, y_test),                       
	steps_per_epoch=len(y_train) // batch_size, epochs=20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/20
15/15 [==============================] - 19s 265ms/step - loss: 1.2428 - accuracy: 0.5683 - val_loss: 0.5247 - val_accuracy: 0.9074
Epoch 2/20
15/15 [==============================] - 3s 177ms/step - loss: 0.4707 - accuracy: 0.8701 - val_loss: 0.2390 - val_accuracy: 0.9259
Epoch 3/20
15/15 [==============================] - 3s 228ms/step - loss: 0.2825 - accuracy: 0.9256 - val_loss: 0.1201 - val_accuracy: 0.9630
Epoch 4/20
15/15 [==============================] - 3s 189ms/step - loss: 0.1945 - accuracy: 0.9445 - val_loss: 0.0802 - val_accuracy: 0.9815
Epoch 5/20
15/15 [==============================] - 3s 208ms/step - loss: 0.1635 - accuracy: 0.9445 - val_loss: 0.0469 - val_accuracy: 0.9907
Epoch 6/20
15/15 [==============================] - 3s 182ms/step - loss: 0.1505 - accuracy: 0.9534 - val_loss: 0.0482 - val_accuracy: 0.9815
Epoch 7/20
15/15 [==============================] - 3s 181ms/step - loss: 0.1463 - accuracy: 0.9434 - val_loss: 0.0378 - val_accuracy: 1.0000
Epoch

# show result of classification with some samples

In [ ]:
import random
i=0
while i<10:
    j=int(random.random()*100)
    sample = x_test[j]
    print(x_test[j].shape)
    sample_show = sample.reshape((100,100,3))
    plt.figure(figsize=(3,3))
    plt.imshow(sample_show , cmap='gray')
    plt.show()
    batch = np.expand_dims(sample, axis=0)
    prediction = model.predict(batch)
    print(prediction)
    print(prediction[0])
    print(   np.argmax(prediction[0])    )  
    i=i+1
    if np.argmax(prediction[0])==0:
      print('blue')
    if np.argmax(prediction[0])==1:
      print('red')
    if np.argmax(prediction[0])==2:
      print('refree')



# calculate accurcy in test dataset

In [ ]:
from sklearn.metrics import accuracy_score
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print(f"accuricy is {acc*100}%")
print(f"loss score is {score}")

2/2 [==============================] - 0s 9ms/step - loss: 0.0049 - accuracy: 1.0000
accuricy is 100.0%
loss score is 0.004909067414700985


# mapping video with classification
in this part we get video of players and start to detect players in each frame and then cut players image and give it to model that trained above after that if it is blue shirt player we add corresponding blue spot in 2d map of mapping video at registered place in 2d map if it's red tishirt player add red spot and if it is refree add yellow spot

In [ ]:
from __future__ import print_function

import cv2
import cv2 as cv
import argparse
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from time import sleep

video_save_path = 'drive/MyDrive/vision project/result.avi'#output video path
arg_input='drive/MyDrive/vision project/output.mp4'#input video path

backSub = cv.createBackgroundSubtractorMOG2()

# backSub = cv.createBackgroundSubtractorKNN()
capture = cv.VideoCapture(cv.samples.findFileOrKeep(arg_input))


if not capture.isOpened():
    print('Unable to open: ' +arg_input)
    exit(0)


##point creation
points1 = np.array([(164, 150),
                    (886, 150),
                    (525, 0),
                    (525, 700)]).astype(np.float32)

points2 = np.array([(149, 165),
                    (1140, 115),
                    (640, 110),
                    (875, 780)]).astype(np.float32)


## hemographic trasnform
H = cv2.getPerspectiveTransform(points2, points1)
print(H)
z=0

#define video
I2 = cv2.imread('2D_field.png')
# w = I2.shape[1]
# h = I2.shape[0]
w=1050
h=700
out = cv2.VideoWriter(video_save_path, cv2.VideoWriter_fourcc(*'MJPG'), 10.0, (w, h))

#start reading image
while True:
    ret, frame = capture.read()
    if frame is None:
        break
    # frame=cv2.GaussianBlur(frame, (3,3), 0);
    fgMask = backSub.apply(frame)
    ret, T = cv2.threshold(fgMask, 254, 255, cv2.THRESH_BINARY)#thsi code for destroy shadows

    ## erosion and dilation
    # erosion_kernel = np.ones((15,5), np.uint8);
    erosion_kernel=np.array([[0 ,0 ,1 ,0 ,0],
                            [0 ,0 ,1 ,1 ,0],
                            [0 ,1 ,1 ,1 ,0],
                            [1, 1, 1, 1, 1],
                            [1, 1, 1, 1, 1],
                            [1, 1, 1, 1, 1],
                            [1, 1, 1, 1, 1],
                            [0, 1, 1, 1, 0],
                            [0, 0, 1, 1, 0],
                            [0 ,0 ,1 ,0 ,0]],np.uint8)
    dilat_kernel = np.array([[0, 0, 1, 0, 0],
                               [0, 0, 1, 1, 0],
                               [0, 1, 1, 1, 0],
                               [1, 1, 1, 1, 1],
                               [1, 1, 1, 1, 1],
                               [1, 1, 1, 1, 1],
                               [1, 1, 1, 1, 1],
                               [0, 1, 1, 1, 0],
                               [0, 0, 1, 1, 0],
                               [0, 0, 1, 0, 0]], np.uint8)
    # dilat_kernel= np.ones((15, 5), np.uint8);
    erosioned = cv2.erode(T, erosion_kernel);
    dilated=cv2.dilate(erosioned,dilat_kernel);

    n2,C2, stats2, centroids2=cv2.connectedComponentsWithStats(dilated)

    #put rules on connected component siza
    delet_array=list()
    n2new=n2
    # print(stats2)
    # print(stats2.shape)
    for i in range(0, n2 ):
        if stats2[i][4] < 150:
           # centroids2
           delet_array.append(i)

           n2new=n2new-1
    centroids2new = np.delete(centroids2, delet_array,0)
    stats2=np.delete(stats2,delet_array, axis=0)

    ##delet connect componet that are not in yard
    # y meghdar avalie centroid ast va x meghdar dovom
    n2new1=n2new
    delet_array=list()
    for i in range(0, n2new - 1):
        if ((1225 - 20) / (60 - 120))*(centroids2new[i][1] - 60) + 1225 > centroids2new[i][0]:
            delet_array.append(i)
            n2new1=n2new1-1

    centroids2new = np.delete(centroids2new, delet_array,0)
    stats2 = np.delete(stats2, delet_array, axis=0)

    ## put text on frames
    cv.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    cv.rectangle(fgMask, (10, 2), (100, 20), (255, 255, 255), -1)
    cv.putText(fgMask, str(n2new+1), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    cv.rectangle(dilated, (10, 2), (100, 20), (255, 255, 255), -1)
    cv.putText(dilated, str(n2new1+1), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))


    ##show images
    cv2.line(frame,(1225, 60),(20, 120),(255,0,255,1),2)


    ## bird eye
    I1 = cv2.imread('drive/MyDrive/vision project/2D_field.png')
    # output_size=(I1.shape[0],I1.shape[1])
    eye_bird=I1
    i=1
    while i<n2new1:
        location = np.array([(centroids2new[i][0].astype(np.int32)),
                   (centroids2new[i][1].astype(np.int32)),
                   (1)])

        cropped_image12 = frame[int(stats2[i][1]):int(stats2[i][1]+stats2[i][3]),int(stats2[i][0]):int(stats2[i][0]+stats2[i][2])]
        print(cropped_image12.shape)
        dim=(100,100)
        cropped_resized = cv2.resize(cropped_image12, dim, interpolation=cv2.INTER_AREA)
        print(cropped_resized.shape)
        batch = np.expand_dims(cropped_resized, axis=0)
        prediction = model.predict(batch)
        print(prediction[0])
        print(   np.argmax(prediction[0])    )

        location=location.transpose()
        location=np.dot(H,location)

        position1=location[0]/location[2]
        position2=location[1]/location[2]
        if np.argmax(prediction[0])==0:
          cv2.circle(eye_bird,(position1.astype(np.int32),position2.astype(np.int32)),10,(0, 0, 255,1),-1);
        if np.argmax(prediction[0])==1:
          cv2.circle(eye_bird,(position1.astype(np.int32),position2.astype(np.int32)),10,(255,0,0,1),-1);
        if np.argmax(prediction[0])==2:
          cv2.circle(eye_bird,(position1.astype(np.int32),position2.astype(np.int32)),10,(255,255,0,1),-1);
        i=i+1
    out.write(eye_bird)
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
out.release()
